# Training and Integrating an Image Classification Model

In this notebook, we will show how to use the `mann` package to train a sparse TensorFlow model and then how to use the `aisquared` package to convert that model and package it into a `.air` file which can be dragged and dropped into the browser.

In [1]:
# First, install dependencies
! pip install mann
! pip install aisquared

## Import packages and preprocess data

In [2]:
# Now that the required packages have been installed, let's import all the required packages
import tensorflow as tf
import aisquared
import mann

In [3]:
# Let's load and preprocess the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# For images, divide all pixel values by 255
x_train = x_train/255
x_test = x_test/255

# Configure the label map
label_map = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck'
]

## Create the model

In [4]:
input_layer = tf.keras.layers.Input(x_train.shape[1:])
x = mann.layers.MaskedConv2D(
    32,
    activation = 'relu'
)(input_layer)
x = mann.layers.MaskedConv2D(
    32,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = mann.layers.MaskedConv2D(
    64,
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    64,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = mann.layers.MaskedConv2D(
    128,
    activation = 'relu'
)(x)
x = mann.layers.MaskedConv2D(
    128,
    activation = 'relu'
)(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Flatten()(x)
x = mann.layers.MaskedDense(512, activation = 'relu')(x)
x = mann.layers.MaskedDense(512, activation = 'relu')(x)
x = mann.layers.MaskedDense(512, activation = 'relu')(x)
output_layer = mann.layers.MaskedDense(10, activation = 'softmax')(x)

model = tf.keras.models.Model(input_layer, output_layer)
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-02-15 15:33:15.995170: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-15 15:33:15.995313: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Sparsify the model initially, then train and apply further sparsification
model = mann.utils.mask_model(
    model,
    40,
    x = x_train[:500],
    y = y_train[:500]
)
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

callback = mann.utils.ActiveSparsification(
    0.65,
    starting_sparsification = 40,
    max_sparsification = 80,
    sparsification_rate = 5
)

model.fit(
    x_train,
    y_train,
    batch_size = 512,
    epochs = 100,
    callbacks = [callback],
    validation_split = 0.2
)

# Remove the training masks from the model
model = mann.utils.remove_layer_masks(model)

model.summary()

2022-02-15 15:33:21.779617: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-15 15:33:21.779800: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/100


2022-02-15 15:33:22.110906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


79/79 [==============================] - ETA: 0s - loss: 2.0509 - accuracy: 0.2384

2022-02-15 15:33:33.306110: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


79/79 [==============================] - 13s 150ms/step - loss: 2.0509 - accuracy: 0.2384 - val_loss: 1.8569 - val_accuracy: 0.3072
Performance measure set to val_accuracy
Model performance has not reached pruning threshold for 1 epoch(s)
Epoch 2/100
79/79 [==============================] - 11s 139ms/step - loss: 1.6747 - accuracy: 0.3795 - val_loss: 1.5179 - val_accuracy: 0.4413
Model performance has not reached pruning threshold for 2 epoch(s)
Epoch 3/100
79/79 [==============================] - 12s 146ms/step - loss: 1.4314 - accuracy: 0.4735 - val_loss: 1.3276 - val_accuracy: 0.5147
Model performance has not reached pruning threshold for 3 epoch(s)
Epoch 4/100
79/79 [==============================] - 11s 139ms/step - loss: 1.2520 - accuracy: 0.5438 - val_loss: 1.1782 - val_accuracy: 0.5802
Model performance has not reached pruning threshold for 4 epoch(s)
Epoch 5/100
79/79 [==============================] - 11s 142ms/step - loss: 1.1411 - accuracy: 0.5861 - val_loss: 1.0926 - val_a

## Check the accuracy of the model on test data and configure the model for deployment

In [6]:
# Check the accuracy on test data
preds = model.predict(x_test).argmax(axis = 1).flatten()
print(f'Accuracy: {(preds == y_test.flatten()).sum()/y_test.shape[0]}')

# Save the model
model.save('cifar10.h5')

2022-02-15 15:37:59.995646: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Accuracy: 0.7478


In [7]:
# Configure the model for integration via the browser

# Harvester
harvester = aisquared.config.harvesting.ImageHarvester()

# Preprocessing steps
resize_step = aisquared.config.preprocessing.Resize([32, 32])
color_step = aisquared.config.preprocessing.ConvertToColor('RGB')
divide_step = aisquared.config.preprocessing.DivideValue(255)

preprocesser = aisquared.config.preprocessing.ImagePreprocessor(
    [
        resize_step,
        color_step,
        divide_step
    ]
)

# Analytic Step - point to the saved model
analytic = aisquared.config.analytic.LocalModel('cifar10.h5', 'cv')

# Postprocessing Step
postprocesser = aisquared.config.postprocessing.MulticlassClassification(label_map)

# Rendering
renderer = aisquared.config.rendering.ImageRendering()

# Put all of the steps together into a configuration object
config = aisquared.config.ModelConfiguration(
    name = 'CIFAR10Classifier',
    harvesting_steps = harvester,
    preprocessing_steps = preprocesser,
    analytic = analytic,
    postprocessing_steps = postprocesser,
    rendering_steps = renderer
)

In [8]:
# Compile the entirety of the configuration and the model into a .air file
config.compile()